In [1]:
from pyspark import SparkContext, SparkConf 
from pyspark.sql import SparkSession
import pyspark.sql as sql
import os

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.14.0 pyspark-shell'

In [2]:
conf = SparkConf().setAppName("6132_Khoroshev_Lab2").setMaster('yarn')

In [3]:
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

In [4]:
posts_sample = spark.read.format('xml')\
.option('rootTag', 'posts')\
.option('rowTag', 'row')\
.load("posts_sample.xml")

In [5]:
posts_sample.printSchema()

root
 |-- _AcceptedAnswerId: long (nullable = true)
 |-- _AnswerCount: long (nullable = true)
 |-- _Body: string (nullable = true)
 |-- _ClosedDate: timestamp (nullable = true)
 |-- _CommentCount: long (nullable = true)
 |-- _CommunityOwnedDate: timestamp (nullable = true)
 |-- _CreationDate: timestamp (nullable = true)
 |-- _FavoriteCount: long (nullable = true)
 |-- _Id: long (nullable = true)
 |-- _LastActivityDate: timestamp (nullable = true)
 |-- _LastEditDate: timestamp (nullable = true)
 |-- _LastEditorDisplayName: string (nullable = true)
 |-- _LastEditorUserId: long (nullable = true)
 |-- _OwnerDisplayName: string (nullable = true)
 |-- _OwnerUserId: long (nullable = true)
 |-- _ParentId: long (nullable = true)
 |-- _PostTypeId: long (nullable = true)
 |-- _Score: long (nullable = true)
 |-- _Tags: string (nullable = true)
 |-- _Title: string (nullable = true)
 |-- _ViewCount: long (nullable = true)



В поле **_Tags** хранится информация о языках<br/>
Из поля **_CreationDate** получаем информацию о дате<br/>
По полю **_ViewCount** будем считать популярность языков

In [6]:
posts = posts_sample.select("_CreationDate", "_ViewCount", "_Tags")
posts.show(2, False)

+-----------------------+----------+------------------------------------------------------+
|_CreationDate          |_ViewCount|_Tags                                                 |
+-----------------------+----------+------------------------------------------------------+
|2008-08-01 02:42:52.667|42817     |<c#><floating-point><type-conversion><double><decimal>|
|2008-08-01 03:08:08.62 |18214     |<html><css><internet-explorer-7>                      |
+-----------------------+----------+------------------------------------------------------+
only showing top 2 rows



In [7]:
from pyspark.sql.functions import col

In [8]:
filtered_posts = posts.filter("_Tags is not null").rdd.map(lambda x: (str(x[0]).split('-')[0], x[1], x[2][1:-1].split("><"))).toDF(["year", "views", "languages"])
filtered_posts = filtered_posts.filter(col("year") >= 2010).filter(col("year") <= 2020)

In [9]:
filtered_posts.show(truncate=False)

+----+-----+---------------------------------------------------------------+
|year|views|languages                                                      |
+----+-----+---------------------------------------------------------------+
|2010|3650 |[c++, character-encoding]                                      |
|2010|617  |[sharepoint, infopath]                                         |
|2010|1315 |[iphone, app-store, in-app-purchase]                           |
|2010|973  |[symfony1, schema, doctrine, fixtures]                         |
|2010|132  |[java]                                                         |
|2010|419  |[visual-studio-2010, stylecop]                                 |
|2010|869  |[cakephp, file-upload, swfupload]                              |
|2010|1303 |[git, cygwin, putty]                                           |
|2010|748  |[drupal, drupal-6]                                             |
|2010|1258 |[php, wordpress, memory]                                       |

Теперь мы третий столбец **languages** распакуем так, чтобы у нас, к примеру, из записи |2010| 599| [c#, c++, java]| получилось 3 новых записи:<br/>
|2010| 599| c#  |<br/>
|2010| 599| c++ |<br/>
|2010| 599| java|<br/>

In [10]:
from pyspark.sql.functions import explode

In [11]:
exploded_posts = filtered_posts.select(filtered_posts.year, filtered_posts.views, explode(filtered_posts.languages)).withColumnRenamed("col", "language") 
exploded_posts.show()

+----+-----+------------------+
|year|views|          language|
+----+-----+------------------+
|2010| 3650|               c++|
|2010| 3650|character-encoding|
|2010|  617|        sharepoint|
|2010|  617|          infopath|
|2010| 1315|            iphone|
|2010| 1315|         app-store|
|2010| 1315|   in-app-purchase|
|2010|  973|          symfony1|
|2010|  973|            schema|
|2010|  973|          doctrine|
|2010|  973|          fixtures|
|2010|  132|              java|
|2010|  419|visual-studio-2010|
|2010|  419|          stylecop|
|2010|  869|           cakephp|
|2010|  869|       file-upload|
|2010|  869|         swfupload|
|2010| 1303|               git|
|2010| 1303|            cygwin|
|2010| 1303|             putty|
+----+-----+------------------+
only showing top 20 rows



In [12]:
from pyspark.sql.functions import lower

languages = spark.read\
.option("header", True)\
.option("inferSchema", True)\
.csv("programming-languages.csv")

languages = languages.drop("wikipedia_url")
languages = languages.withColumn("name",  lower("name"))

languages.show()

+------------+
|        name|
+------------+
|     a# .net|
|  a# (axiom)|
|  a-0 system|
|          a+|
|         a++|
|        abap|
|         abc|
|   abc algol|
|       abset|
|       absys|
|         acc|
|      accent|
|    ace dasl|
|        acl2|
|     act-iii|
|     action!|
|actionscript|
|         ada|
|     adenine|
|        agda|
+------------+
only showing top 20 rows



In [13]:
exploded_posts = exploded_posts.crossJoin(languages).where("language = name")
counted_posts = exploded_posts.groupBy("year", "language").agg({"views": "sum"})
counted_posts.show()

+----+-----------+----------+
|year|   language|sum(views)|
+----+-----------+----------+
|2013|     erlang|      2302|
|2017| typescript|     29031|
|2017|        sed|        93|
|2013| javascript|    609571|
|2013|         f#|      4317|
|2012| powershell|     17311|
|2019|        php|      3753|
|2017|    haskell|      4040|
|2013| autohotkey|      4421|
|2013|applescript|      2059|
|2019|      xpath|        27|
|2015|     racket|       762|
|2017|         go|      1356|
|2018|      perl6|       109|
|2015|       dart|       402|
|2015|       rust|       117|
|2012|         f#|      1222|
|2018|     python|     99996|
|2017|     prolog|        76|
|2016|      latex|       461|
+----+-----------+----------+
only showing top 20 rows



In [14]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

window = Window.partitionBy(counted_posts.year).orderBy(col("sum(views)").desc())
tempDF = counted_posts.withColumn("rank", row_number().over(window))
tempDF = tempDF.withColumnRenamed("sum(views)", "views")
tempDF.show()

+----+-----------+------+----+
|year|   language| views|rank|
+----+-----------+------+----+
|2016| javascript|233802|   1|
|2016|     python|219510|   2|
|2016|       java|169513|   3|
|2016|        php|105849|   4|
|2016|          r| 49717|   5|
|2016| powershell| 36175|   6|
|2016| typescript| 33343|   7|
|2016|          c| 17568|   8|
|2016|         go| 13062|   9|
|2016|       ruby| 12909|  10|
|2016|      scala| 11725|  11|
|2016|       curl|  9386|  12|
|2016|       bash|  7537|  13|
|2016|       nasm|  6844|  14|
|2016|objective-c|  5722|  15|
|2016|     groovy|  4051|  16|
|2016|     matlab|  3743|  17|
|2016|       chef|  2009|  18|
|2016|        awk|  1808|  19|
|2016|    haskell|  1631|  20|
+----+-----------+------+----+
only showing top 20 rows



In [15]:
tempDF = tempDF.filter(col("rank") <= 10)
result = tempDF.drop("rank")
result = result.orderBy(col("year").asc(), col("views").desc())
result.show()

+----+-----------+-------+
|year|   language|  views|
+----+-----------+-------+
|2010|        php|1189629|
|2010|       java| 563211|
|2010| javascript| 316131|
|2010|objective-c|  97009|
|2010|       ruby|  76215|
|2010|          c|  66587|
|2010|     python|  60672|
|2010|     matlab|  51865|
|2010|applescript|  32305|
|2010|     delphi|  13065|
|2011| javascript| 809078|
|2011|       java| 389834|
|2011|        php| 246770|
|2011|          c| 238277|
|2011|objective-c| 218934|
|2011|     python| 203180|
|2011|       bash|  60805|
|2011|       ruby|  39223|
|2011|       perl|  28502|
|2011|     matlab|  18816|
+----+-----------+-------+
only showing top 20 rows



In [17]:
result.write.mode('overwrite').parquet("top_10_languages_per_year_from_2010_to_2020.parquet")

In [22]:
!hadoop fs -get "top_10_languages_per_year_from_2010_to_2020.parquet"  .

In [25]:
!ls

6132_Khoroshev_Lab1.ipynb
6132_Khoroshev_Lab2.ipynb
data
L1_interactive_bike_analysis_python_with_dataframes.ipynb
L1_interactive_bike_analysis_python_with_rdd.ipynb
top_10_languages_per_year_from_2010_to_2020.parquet
